In [111]:
import akshare as ak
import pandas as pd



['600988', '688671', '600571', '600360', '603132', '600501', '600531', '688331', '603389', '600418', '600227', '603079']
      序号      代码     名称    最新价   涨跌幅   涨跌额        成交量           成交额     振幅  \
87    88  600988   赤峰黄金  18.14  4.67  0.81   512866.0  9.128946e+08   6.58   
99   100  688671   碧兴物联  19.22  4.46  0.82    12545.0  2.402518e+07   8.70   
108  109  600571    信雅达   9.98  4.28  0.41   311766.0  3.112785e+08  11.08   
111  112  600360   ST华微   2.94  4.26  0.12   390394.0  1.125418e+08   6.38   
121  122  603132   金徽股份  14.70  4.11  0.58    52742.0  7.627377e+07   5.45   
124  125  600501   航天晨光  14.14  4.05  0.55   366313.0  5.071982e+08   8.68   
146  147  600531  XD豫光金   6.53  3.65  0.23   360341.0  2.304304e+08   5.40   
152  153  688331   荣昌生物  39.94  3.61  1.39    55209.0  2.130767e+08   7.57   
163  164  603389   亚振家居   4.74  3.49  0.16   171578.0  8.198577e+07  11.35   
186  187  600418   江淮汽车  17.28  3.23  0.54  1080650.0  1.834268e+09   8.06   
187  188  600227    赤

In [ ]:
from datetime import datetime, timedelta

def get_filtered_stocks(start_date_str, end_date_str):
    """
    获取在指定日期范围内，符合成交量条件的股票数据。
    
    参数:
    start_date_str (str): 开始日期，格式为"YYYYMMDD"
    end_date_str (str): 结束日期，格式为"YYYYMMDD"
    
    返回:
    DataFrame: 符合条件的股票数据
    """
    
    def get_previous_week_avg_volume(stock_code, start_date, end_date):
        """获取指定股票在给定日期范围内的日线数据，并计算之前一周的平均成交量"""
        try:
            stock_data = ak.stock_zh_a_hist(symbol=stock_code, period="daily", start_date=start_date, end_date=end_date, adjust="")
            if stock_data.empty or '成交量' not in stock_data.columns:
                return None
            
            stock_data['成交量'] = pd.to_numeric(stock_data['成交量'], errors='coerce')
            stock_data['之前一周平均成交量'] = stock_data['成交量'].rolling(window=7).mean()
            return stock_data

        except Exception as e:
            print(f"Failed to fetch data for stock {stock_code}: {str(e)}")
            return None
    
    # 获取主板A股的股票信息
    stock_info_sh_name_code_df = ak.stock_info_sh_name_code(symbol="主板A股")
    all_stock_codes = stock_info_sh_name_code_df['证券代码'].tolist()

    # 初始化一个空的DataFrame，用于存储符合条件的股票数据
    filtered_stocks = pd.DataFrame(columns=stock_info_sh_name_code_df.columns)

    # 获取当天的股票数据
    stock_sh_a_spot_em_df = ak.stock_sh_a_spot_em()

    # 筛选出涨跌幅在3%到5%之间，量比大于1，换手率大于3%的股票代码
    current_stock_list = stock_sh_a_spot_em_df.loc[
        (stock_sh_a_spot_em_df["涨跌幅"] > 3) &
        (stock_sh_a_spot_em_df["涨跌幅"] < 5) &
        (stock_sh_a_spot_em_df["量比"] > 1) &
        (stock_sh_a_spot_em_df["换手率"] > 3)
    ]

    # 获取筛选后的股票代码列表
    stock_codes = current_stock_list["代码"].tolist()

    # 将开始和结束日期转换为 datetime 对象
    end_date = datetime.strptime(end_date_str, "%Y%m%d")
    start_date = datetime.strptime(start_date_str, "%Y%m%d")

    # 获取之前一周的开始和结束日期
    previous_week_end_date = start_date - timedelta(days=1)
    previous_week_start_date = previous_week_end_date - timedelta(days=7)
    previous_week_start_date_str = previous_week_start_date.strftime("%Y%m%d")
    previous_week_end_date_str = previous_week_end_date.strftime("%Y%m%d")

    # 遍历每一个股票代码，进行过滤
    for stock_code in all_stock_codes:
        try:
            # 获取该股票最近七天的历史数据
            stock_data = ak.stock_zh_a_hist(symbol=stock_code, period="daily", start_date=start_date_str, end_date=end_date_str, adjust="")
            
            # 如果历史数据为空或者成交量数据为空，则跳过该股票
            if stock_data.empty or '成交量' not in stock_data.columns:
                continue
            
            # 将成交量转换为数值类型
            stock_data['成交量'] = pd.to_numeric(stock_data['成交量'], errors='coerce')
            
            # 获取之前一周的平均成交量
            previous_week_data = get_previous_week_avg_volume(stock_code, previous_week_start_date_str, previous_week_end_date_str)
            if previous_week_data is None or previous_week_data['之前一周平均成交量'].isna().all():
                continue
            
            previous_week_avg_volume = previous_week_data['之前一周平均成交量'].iloc[-1]
            
            # 找出符合条件的数据
            filtered_data = stock_data[
                (stock_data['成交量'] > stock_data['成交量'].shift(1) * 2) & 
                (stock_data['成交量'].shift(-1) > stock_data['成交量']) &
                (stock_data['成交量'].shift(1) > previous_week_avg_volume * 1.2)
            ].dropna()
            
            # 如果符合条件的数据不为空，并且股票代码在今日选定的股票列表中
            if not filtered_data.empty and stock_code in stock_codes:
                filtered_stocks = pd.concat([filtered_stocks, stock_info_sh_name_code_df[stock_info_sh_name_code_df['证券代码'] == stock_code]])
        
        except Exception as e:
            print(f"Failed to fetch data for stock {stock_code}: {str(e)}")

    return filtered_stocks


In [136]:
# 示例调用
start_date_str = "20240625"
end_date_str = "20240702"
filtered_stocks = get_filtered_stocks(start_date_str, end_date_str)
print(filtered_stocks)

NameError: name 'get_filtered_stocks' is not defined